# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,422.533358,0.456155,1234,422.989513,422.077204,845.979026,844.154407
6,243.219529,0.660638,1234,243.880167,242.558892,487.760334,485.117784
7,143.582021,-0.590663,1234,142.991358,144.172685,285.982716,288.345370
8,584.831994,0.215238,1234,585.047232,584.616756,1170.094464,1169.233512
9,522.524940,0.867685,1234,523.392625,521.657255,1046.785251,1043.314509
10,452.388026,0.656065,1234,453.044091,451.731961,906.088183,903.463922
11,958.078871,0.007854,1234,958.086725,958.071017,1916.173450,1916.142035
12,908.075677,-0.830719,1234,907.244958,908.906396,1814.489916,1817.812792
13,881.814111,-0.534236,1234,881.279875,882.348347,1762.559749,1764.696694
14,728.531894,-0.667524,1234,727.864370,729.199418,1455.728740,1458.398836


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-422.989513,422.077204
6,-243.880167,242.558892
7,-142.991358,144.172685
8,-585.047232,584.616756
9,-523.392625,521.657255
10,-453.044091,451.731961
11,-958.086725,958.071017
12,-907.244958,908.906396
13,-881.279875,882.348347
14,-727.864370,729.199418


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-422.989513,422.077204
6,-243.880167,242.558892
7,-142.991358,144.172685
8,-585.047232,584.616756
9,-523.392625,521.657255
10,-453.044091,451.731961
11,-958.086725,958.071017
12,-907.244958,908.906396
13,-881.279875,882.348347
14,-727.864370,729.199418


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-422.98951317,  422.07720372],
       [-243.88016705,  242.55889182],
       [-142.99135803,  144.17268485],
       [-585.04723185,  584.61675586],
       [-523.39262533,  521.65725463],
       [-453.04409144,  451.73196101],
       [-958.08672514,  958.07101747],
       [-907.24495824,  908.90639618],
       [-881.27987464,  882.34834675],
       [-727.86437008,  729.19941793]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,843.979026,-844.154407
6,485.760334,-485.117784
7,283.982716,-288.345370
8,1168.094464,-1169.233512
9,1044.785251,-1043.314509
10,904.088183,-903.463922
11,1914.173450,-1916.142035
12,1812.489916,-1817.812792
13,1760.559749,-1764.696694
14,1453.728740,-1458.398836


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.422533,0.728077
6,0.243220,0.830319
7,0.143582,0.204668
8,0.584832,0.607619
9,0.522525,0.933843
10,0.452388,0.828033
11,0.958079,0.503927
12,0.908076,0.084641
13,0.881814,0.232882
14,0.728532,0.166238


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,422.533358,0.456155
6,243.219529,0.660638
7,143.582021,-0.590663
8,584.831994,0.215238
9,522.524940,0.867685
10,452.388026,0.656065
11,958.078871,0.007854
12,908.075677,-0.830719
13,881.814111,-0.534236
14,728.531894,-0.667524


In [23]:
# Check that the data is the same
data2 = vocs.denormalize_inputs(vocs.normalize_inputs(data)) 
for k in data2:
    print(k, np.allclose(data2[k], data[k]))

a True
b True


# Error handling

In [ ]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [ ]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

In [ ]:
data['a']  = np.nan

In [ ]:
a = 2
def f(x=a):
    return x
a=99
f()

In [ ]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

In [ ]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)